In [1]:
!pip install -qU pip awscli boto3 sagemaker 
!pip install nvidia-pyindex
!pip install tritonclient[http]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com


In [6]:
import os

In [ ]:
import boto3
import json
import sagemaker
import time
import os
from sagemaker import get_execution_role

sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime")

In [ ]:
triton_image_uri = "354625738399.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tritonserver:22.05-py"

## Package model files and uploading to S3

In [5]:
!wget -q -P triton-serve-fil/preprocessing https://rapidsai-data.s3.us-east-2.amazonaws.com/conda-pack/rapidsai/rapids22.04_cuda11.5_py3.8.tar.gz

In [ ]:
!tar -C triton-serve-fil/ -czf model.tar.gz preprocessing
model_uri = sagemaker_session.upload_data(path="model.tar.gz", key_prefix="triton-serve")

**TODO**: Note the reshape we had to do for output here. This is specific to this example.

In [12]:
USE_GPU = True
MODEL_DIR = './triton-serve/fil'

# Maximum size in bytes for input and output arrays
MAX_MEMORY_BYTES = 60_000_000
NUM_FEATURES = 15
NUM_CLASSES = 2
bytes_per_sample = (NUM_FEATURES + NUM_CLASSES) * 4
max_batch_size = MAX_MEMORY_BYTES // bytes_per_sample

IS_CLASSIFIER = True
model_format = 'xgboost_json'

# Select deployment hardware (GPU or CPU)
if USE_GPU:
    instance_kind = 'KIND_GPU'
else:
    instance_kind = 'KIND_CPU'

# whether the model is doing classification or regression    
if IS_CLASSIFIER:
    classifier_string = 'true'
else:
    classifier_string = 'false'

# whether to predict probabilites or not
predict_proba = False

if predict_proba:
    predict_proba_string = 'true'
else:
    predict_proba_string = 'false'

config_text = f"""backend: "fil"
max_batch_size: {max_batch_size}
input [                                 
 {{  
    name: "input__0"
    data_type: TYPE_FP32
    dims: [ {NUM_FEATURES} ]                    
  }} 
]
output [
 {{
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1 ]
    reshape: {{ shape: [ 1 ] }}
  }}
]
instance_group [{{ kind: {instance_kind} }}]
parameters [
  {{
    key: "model_type"
    value: {{ string_value: "{model_format}" }}
  }},
  {{
    key: "predict_proba"
    value: {{ string_value: "{predict_proba_string}" }}
  }},
  {{
    key: "output_class"
    value: {{ string_value: "{classifier_string}" }}
  }},
  {{
    key: "threshold"
    value: {{ string_value: "0.5" }}
  }},
  {{
    key: "storage_type"
    value: {{ string_value: "AUTO" }}
  }}
]

dynamic_batching {{}}"""

config_path = os.path.join(MODEL_DIR, 'config.pbtxt')
with open(config_path, 'w') as file_:
    file_.write(config_text)

## Run Inference

In [10]:
import pandas as pd

In [11]:
data_infer = pd.read_csv("data_infer.csv")

In [18]:
type(sample_data)

numpy.ndarray

In [ ]:
payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": list(sample_data.shape), "datatype": "STRING", "data": sample_data},
    ]
}